In [1]:
import sqlite3
import numpy as np

In [2]:
conn = sqlite3.connect("news-db.sqlite")

In [27]:
def get_user_dicts(cur, corpus_lengths):
  user_ids = [elt[0] for elt in cur.execute("SELECT DISTINCT user_id from session_log").fetchall()]
  user_data = {user: [elt for elt in cur.execute(f"SELECT article_id, time_spent, rating FROM session_log WHERE user_id == {0} AND rating IS NOT NULL GROUP BY article_id").fetchall()] for user in user_ids}
  user_doc_dict = {id:[elt[0] for elt in user_data[id]] for id in user_ids}
  user_rating_dict = {id:[get_engagement(elt[1], corpus_lengths[elt[0]], elt[2]) for elt in user_data[id]] for id in user_ids}

  return user_doc_dict, user_rating_dict

In [28]:
def get_engagement(time, num_words, click, mean = 445, var = 60):
    if click in [-1, 1]:
        return click
    else:
        mean_t, std_t  = (1/mean * num_words) * 60, np.sqrt((1/var * num_words) * 60) # reading time in seconds 
        amp  = np.random.uniform(0.85, 0.95)
        
        if time <= mean_t: return amp * pow(np.e, -(time - mean_t)**2/(2 * std_t**2))
        elif (mean_t < time) and time < (mean_t + 2 * std_t): return amp
        else: return 0


In [29]:
corpus = [elt[0] for elt in list(conn.cursor().execute("SELECT content from articles_processed")) if not(elt[0] is None)]
# note document lengths for estimating average reading time
corpus_lengths = [len(doc) for doc in corpus]

In [ ]:
conn.cursor().execute("SELECT content from articles_processed")